In [15]:
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions
nltk.download('stopwords')
nltk.download('wordnet')

from typing import List

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\htc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\htc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Data

In [2]:
file_path = r"E:\DATA SCIENCE\NLP-Tea\Data\IMDB Dataset Movie Reviews\IMDB Dataset.csv"
df = pd.read_csv(file_path)

df.head() #  (50000, 2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
text_data = list(df["review"][:1000]) # First 1000 Row Only 
text_data[:5]

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [4]:
html_text = "'A wonderful little production. <br /><br />The filming technique is very unassuming-"
html_text = html_text.lower()
clean_text = re.sub(r'<[^>]+>', '', html_text)
print(clean_text)

'a wonderful little production. the filming technique is very unassuming-


## Preprocessing

In [42]:
def preprocessing(text: str) -> list[str]:
    """
    Preprocesses a given text:
    - Lowercases text
    - Contraction Handling
    - Removes punctuation and digits
    - Removes stopwords
    - Tokenizes into words
    - Applies lemmatization or stemming

    Args:
        document (str): The raw input text

    Returns:
        List of str: Cleaned and preprocessed text
    """

    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    # Convert Text to Lowercase (Normalization)
    text_lower = text.lower()
    text_no_tags = re.sub(r'<[^>]+>', '', text_lower)

    # Contraction Handling
    text_no_tags = contractions.fix(text_no_tags)

    # Removing Punctuation
    text_no_punct = re.sub(r'[^a-zA-Z\s]', '', text_no_tags) # \' for keep apostrophes (e.g. don't, it's)


    # 3. Tokens
    tokens = re.split(r"\s+", text_no_punct) 
    tokens = [t for t in tokens if t]
    # or use nltk tokenizer
    tokens = word_tokenize(text_no_punct)

    # 4. Stop word removal
    filtered_tokens  = [token for token in tokens if token not in stop_words]

    # 5. Lemmatization 
    lemma_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens ]
    # or stemmer
    stemm_tokens = [stemmer.stem(token) for token in filtered_tokens ]

    return lemma_tokens

text_data = list(df["review"][:100]) # First 100 Row Only
preprocessed_text = [preprocessing(text) for text in text_data]
print(preprocessed_text[:3])

[['one', 'reviewer', 'mentioned', 'watching', 'oz', 'episode', 'hooked', 'right', 'exactly', 'happened', 'methe', 'first', 'thing', 'struck', 'oz', 'brutality', 'unflinching', 'scene', 'violence', 'set', 'right', 'word', 'go', 'trust', 'show', 'faint', 'hearted', 'timid', 'show', 'pull', 'punch', 'regard', 'drug', 'sex', 'violence', 'hardcore', 'classic', 'use', 'wordit', 'called', 'oz', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focus', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cell', 'glass', 'front', 'face', 'inwards', 'privacy', 'high', 'agenda', 'city', 'home', 'manyaryans', 'muslim', 'gangsta', 'latino', 'christian', 'italian', 'irish', 'moreso', 'scuffle', 'death', 'stare', 'dodgy', 'dealing', 'shady', 'agreement', 'never', 'far', 'awayi', 'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'go', 'show', 'would', 'dare', 'forget', 'pretty', 'picture', 'painted', 'mainstream', 'audience', 'forget', 'charm', 'forget

## From Scratch

In [86]:
all_tokens =[]
for lst_tokens in preprocessed_text:
    all_tokens.extend(lst_tokens)
vocab = sorted(set(all_tokens)) # Unique Words


def TermFrequency(term: str, doc: list[str]) :
    tf = 0
    if term not in doc:
        return tf
    for t in doc:
        if t == term:
            tf+=1
    return tf 
    


def BagOfWords(vocab: list, preprocessed_text: list[list]) -> np.ndarray :

    n_docs = len(preprocessed_text)
    n_vocab = len(vocab)
    
    bow_matrix = np.zeros(shape=(n_docs, n_vocab)) # (# documents, # vocabulary words)

    for doc_idx, doc in enumerate(preprocessed_text): # For Each Document 

        bow_vec = np.zeros(shape=n_vocab)
        #  For each word in the vocabulary, calculate its term frequency in this document
        for term_idx,term in enumerate(vocab): 
                tf = TermFrequency(term=term, doc=doc)
                bow_vec[term_idx] = tf

        bow_matrix[doc_idx] = bow_vec
    return bow_matrix
bag_of_word_matrix = BagOfWords(vocab, preprocessed_text)

In [77]:
print(bag_of_word_matrix[3][400:700])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0.]


## Built in

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = []
for p in preprocessed_text:
    corpus.append(" ".join(p))

vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(corpus)
bow_dense = bow_matrix.toarray()

vocab = vectorizer.get_feature_names_out()
print("Vocabulary:", vocab)
print("Bag of Words Matrix:\n", bow_dense)


Vocabulary: ['abbot' 'abbreviated' 'abetted' ... 'zoo' 'zoom' 'zwick']
Bag of Words Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [89]:
bow_dense.shape

(100, 4438)